In [1]:
import numpy as np

In [2]:
with open('./input.txt', 'r') as f:
    puzzle_input = f.read()
    
with open('./test_input.txt', 'r') as f:
    puzzle_test_input = f.read()
    
def process_input(_input):
    return [ [ int(c) for c in row ] for row in _input.split('\n') ]

In [73]:
def create_heightmap(_input):
    """Padding to avoid out of range errors."""
    return np.pad(np.array(process_input(_input)), [(1,1),(1,1)], mode='constant', constant_values=-1)

# Part 1

In [81]:
def get_neighbors(point, heightmap):
    y,x = point
    return [ (point, heightmap[point]) for point in [(y-1, x), (y+1, x), (y, x-1), (y, x+1)] ]

In [82]:
def find_low_points(_input):
    heightmap = create_heightmap(_input)
    low_points = []
    for row_index, row in enumerate(heightmap):
        if row_index == 0 or row_index == len(heightmap) - 1:
            # Skip row padding.
            continue
        for col_index, value in enumerate(row):
            if col_index == 0 or col_index == len(row) - 1:
                # Skip column padding.
                continue
            
            # get neighbors and ignore numpy padding cells
            neighbors = list(filter(lambda p: p[1] != -1, get_neighbors((row_index, col_index), heightmap)))
            if min(neighbors, key=lambda p: p[1])[1] > value:
                # we found a low point
                low_points.append(((row_index, col_index), value))
    return low_points

In [87]:
sum([ point[1] + 1 for point in find_low_points(puzzle_input) ])

458

# Part 2

In [84]:
def search_basin(visited_points, node, heightmap):
    node_position, node_value = node
    
    # Mark this location as visited
    visited_points[node_position] = True
    neighbor_points = get_neighbors(node_position, heightmap)
    for (position, value) in neighbor_points:
        if position not in visited_points:
            # We haven't visited this point
            if value not in (-1, 9) and value > node_value:
                # This point is a valid part of the basin, let's check it out!
                search_basin(visited_points, (position, value), heightmap)

In [85]:
heightmap = create_heightmap(puzzle_input)
low_points = find_low_points(puzzle_input)
basins = []
for low_point in low_points:
    visited_points = {}
    search_basin(visited_points, low_point, heightmap)
    basins.append({ k:v for k,v in visited_points.items() if v})

# Sort the basins by length descending and take the product of the top 3
np.prod([ len(basin) for basin in sorted(basins, key=lambda basin: -len(basin))[:3] ])

1391940